Скачиваем геном бактерии T. oleivorans MIL-1

In [1]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=/root/edirect:\${PATH}" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${HOME}/edirect:${PATH}



In [2]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

Устанавливаем программы

In [3]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 74.3 MB/s eta 0:00:00


In [4]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,654 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:14 http://archi

In [5]:
!apt-get install ncbi-blast+

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  liblmdb0 ncbi-data
The following NEW packages will be installed:
  liblmdb0 ncbi-blast+ ncbi-data
0 upgraded, 3 newly installed, 0 to remove and 50 not upgraded.
Need to get 15.9 MB of archives.
After this operation, 71.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblmdb0 amd64 0.9.24-1build2 [47.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncbi-data all 6.1.20170106+dfsg1-9 [3,519 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ncbi-blast+ amd64 2.12.0+ds-3build1 [12.3 MB]
Fetched 15.9 MB in 1s (12.6 MB/s)
Selecting previously unselected package liblmdb0:amd64.
(Reading database ... 123622 files and directories currently installed.)
Preparing to unpack .../liblmdb0_0.9.24-1build2_amd64.deb ...
Unpacking liblmdb0:amd64 (0.9.24-1build2) ...
Selec

## Предсказание рибосомальной рнк

загрузила файлы scaffolds.hits_from_MIL_1.txt и platanus_gapClosed.fa

Читаем файл T_oleivorans_MIL_1.gbk, а также находим начало, конец и знак стренда в рРНК

In [6]:
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation
import re

In [7]:
path = "T_oleivorans_MIL_1.gbk"
coords_start = []
coords_end = []
coords_st = []
data = SeqIO.read("T_oleivorans_MIL_1.gbk", "genbank")
display(data)
for f in SeqIO.read(path, "genbank").features:
  if f.type == 'rRNA':
    coords_start.append(int(f.location.start))
    coords_end.append(int(f.location.end))
    coords_st.append(f.strand)
print("start", coords_start)
print("end", coords_end)
print("st", coords_st)

SeqRecord(seq=Seq('TTGGGTAGAATGGCTGGCTCGATTGAAATCTTTGATGACGCTAGTAACAGGGGT...CTC'), id='HF680312.1', name='HF680312', description='Thalassolituus oleivorans MIL-1 complete genome', dbxrefs=['BioProject:PRJEB1425', 'BioSample:SAMEA2272589'])

start [341494, 343487, 346568, 2580484, 2580781, 2583918, 3418645, 3418942, 3422079, 3423993, 3424290, 3427427]
end [343033, 346374, 346684, 2580600, 2583668, 2585457, 3418761, 3421829, 3423618, 3424109, 3427177, 3428966]
st [1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


/usr/local/lib/python3.10/dist-packages/Bio/SeqFeature.py:231: BiopythonDeprecationWarning: Please use .location.strand rather than .strand
  warnings.warn(


Формируем файл FASTA, содержащий последовательности рРНК, найденные в геноме, с соответствующими заголовками

In [8]:
f = open('resSeq.fasta', 'w')
for i in range(len(coords_st)):
  f.write(f'> rRNA {coords_start[i]}...{coords_end[i]}\n')
  f.write(str(data.seq[coords_start[i]:coords_end[i]]) + '\n')
f.close()

Выполняем BLAST - сравнениваем последовательности рРНК, сохранённых в файле resSeq.fasta, с последовательностями, содержащимися в файле с собранным геномом

In [9]:
!blastn -query resSeq.fasta -subject scaffolds-3.fasta > Data_inform.gbk

Читаем Data_inform.gbk, содержащий результаты BLAST, и извлекаем информацию о совпадениях рРНК с последовательностями в нашем геноме

In [10]:
path = "Data_inform.gbk"

with open(path, 'r') as file:
    search_mode = False
    sequences = []
    similarity_scores = []
    current_similarity = []
    markers = ["Query=", ">", "Identities"]

    for line in file:
        parts = line.split()

        if parts:
            if parts[0] == markers[0]:
                if search_mode:
                    similarity_scores.append(current_similarity)
                    print(f'Similarity for rRNA {query_name}')
                    for i, seq in enumerate(sequences):
                        print(f"Sequence = {seq}, Similarity = {similarity_scores[i]}")
                query_name = parts[2]
                search_mode = True
                sequences = []
                similarity_scores = []
                current_similarity = []

            elif parts[0] == markers[1]:
                if sequences:
                    similarity_scores.append(current_similarity)
                sequences.append(parts[1])
                current_similarity = []

            elif parts[0] == markers[2]:
                current_similarity.append(parts[3][1:-2])

    if search_mode:
        similarity_scores.append(current_similarity)
        print(f'Similarity for rRNA {query_name}')
        for i, seq in enumerate(sequences):
            print(f"Sequence = {seq}, Similarity = {similarity_scores[i]}")


Similarity for rRNA 341494...343033
Sequence = scaffold73_cov665, Similarity = ['100%']
Sequence = scaffold3_cov273, Similarity = ['100%']
Similarity for rRNA 343487...346374
Sequence = scaffold69_cov665, Similarity = ['100%']
Sequence = scaffold68_cov665, Similarity = ['100%']
Sequence = scaffold70_cov714, Similarity = ['100%']
Similarity for rRNA 346568...346684
Sequence = scaffold69_cov665, Similarity = ['100%']
Sequence = scaffold3_cov273, Similarity = ['100%']
Similarity for rRNA 2580484...2580600
Sequence = scaffold69_cov665, Similarity = ['99%']
Sequence = scaffold3_cov273, Similarity = ['97%']
Similarity for rRNA 2580781...2583668
Sequence = scaffold69_cov665, Similarity = ['100%']
Sequence = scaffold68_cov665, Similarity = ['100%']
Sequence = scaffold70_cov714, Similarity = ['100%']
Similarity for rRNA 2583918...2585457
Sequence = scaffold73_cov665, Similarity = ['99%']
Sequence = scaffold3_cov273, Similarity = ['100%']
Similarity for rRNA 3418645...3418761
Sequence = scaffold